In [1]:
import helper as hlp
kaggle_dataset = 'data/kaggle_dataset/Pakistan_Largest_Ecommerce_Dataset.csv'
preprocessed_dataset = 'data/preprocessed_dataset/preprocessed_dataset.csv'
hlp.preprocess_dataset(kaggle_dataset, preprocessed_dataset)


/Users/egebilge/Developer/GitHub/E-commerce-Cart-Abandonment-Prediction/helper.py:6: DtypeWarning: Columns (1,2,3,7,8,9,11,12,13,14,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file)


Preprocessed dataset saved to data/preprocessed_dataset/preprocessed_dataset.csv
